In [1]:
def return_data(fold, month, with_scaling):
    train_input = pd.read_csv(
        "../data/beijing-18/time_feature"
        + "/fold"
        + str(fold)
        + "/train_data_"
        + month
        + ".csv.gz"
    )
    test_input = pd.read_csv(
        "../data/beijing-18/time_feature"
        + "/fold"
        + str(fold)
        + "/test_data_"
        + month
        + ".csv.gz"
    )
    test_output = np.array(test_input["PM25_Concentration"])
    train_output = np.array(train_input["PM25_Concentration"])
    train_input = train_input.drop(
        ["station_id", "PM25_Concentration", "time", "filled"], axis=1
    )
    try:
        test_input = test_input.drop(
            ["PM25_Concentration", "station_id", "time", "filled"], axis=1
        )
    except:
        test_input = test_input.drop(["station_id", "time", "filled"], axis=1)
    #     test_output= test_output.drop(['time'],axis=1)
    if with_scaling:
        scaler = MinMaxScaler().fit(train_input)
        train_input = scaler.transform(train_input)
        test_input = scaler.transform(test_input)
    return train_input, train_output, test_input, test_output


def run_model(model, train_input, train_output, test_input, test_output, ret_output):
    model.fit(np.array(train_input), train_output)
    test_pred = model.predict(np.array(test_input))
    err = mean_squared_error(test_pred, test_output, squared=False)
    mae = mean_absolute_error(test_pred, test_output)
    r2 = r2_score(test_pred, test_output)
    if ret_output:
        return (err, mae, r2), test_pred
    else:
        return (err, mae, r2)

In [2]:
mean_rmse, mean_mae, mean_r2 = 0, 0, 0

In [3]:
X_train, y_train, X_test, y_test = return_data(fold=0, month="mar", with_scaling=False)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 0)

lgb_train = lgb.Dataset(X_train, y_train)

gbm = lgb.train({}, lgb_train, 3000, callbacks=[], verbose_eval=200)

NameError: name 'pd' is not defined

In [ ]:
gbm.best_iteration

0

In [ ]:
y_pred = gbm.predict(X_test, num_iteration=2889)

rmse = mean_squared_error(y_pred, y_test, squared=False)
mae = mean_absolute_error(y_pred, y_test)
r2 = r2_score(y_pred, y_test)

print("RMSE: ", rmse)
print("MAE: ", mae)
print("R2 score: ", r2)
mean_rmse += rmse
mean_mae += mae
mean_r2 += r2

RMSE:  28.877441820241287
MAE:  17.13302233015443
R2 score:  0.853380022736713


In [ ]:
for fold in [0, 1, 2]:
    X_train, y_train, X_test, y_test = return_data(
        fold=fold, month="mar", with_scaling=False
    )
    lgb_train = lgb.Dataset(X_train, y_train)
    gbm = lgb.train({"boosting": "dart"}, lgb_train, 3000)
    y_pred = gbm.predict(X_test, num_iteration=2889)

    rmse = mean_squared_error(y_pred, y_test, squared=False)
    mae = mean_absolute_error(y_pred, y_test)
    r2 = r2_score(y_pred, y_test)

    print("RMSE: ", rmse)
    print("MAE: ", mae)
    print("R2 score: ", r2)
    mean_rmse += rmse
    mean_mae += mae
    mean_r2 += r2

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.090192 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 822
[LightGBM] [Info] Number of data points in the train set: 14880, number of used features: 22
[LightGBM] [Info] Start training from score 88.651008
RMSE:  28.877441820241287
MAE:  17.13302233015443
R2 score:  0.853380022736713
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000848 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 819
[LightGBM] [Info] Number of data points in the train set: 15624, number of used features: 22
[LightGBM] [Info] Start training from score 92.397849
RMSE:  26.97994274651366
MAE:  16.174745195499895
R2 score:  0.8662637407567324
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000904 seconds.
You can set `force_col_wise=true` to remove 

In [ ]:
np.array([mean_rmse, mean_mae, mean_r2]) / 3

array([38.40700317, 22.5077261 ,  1.13260311])